In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('Task 3 and 4_Loan_Data.csv')

In [4]:
df.head()


,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0


In [15]:
fico = np.array([df.iloc[:,-2]])
fico = fico.reshape(10000)

In [16]:
fico

array([605, 572, 602, ..., 596, 647, 757], dtype=int64)

In [17]:
def create_rating_map(fico_scores, num_buckets):
    # Step 1: Determine the range of FICO scores
    min_score = np.min(fico_scores)
    max_score = np.max(fico_scores)
    
    # Step 2: Calculate the bucket width
    bucket_width = (max_score - min_score) / num_buckets
    
    # Step 3: Define the rating labels
    rating_labels = ["Excellent", "Very Good", "Good", "Fair", "Poor"]
    
    # Step 4: Map FICO scores to rating buckets
    rating_map = {}
    for score in fico_scores:
        # Calculate the bucket index
        bucket_index = int((score - min_score) / bucket_width)
        # Ensure the bucket index stays within the valid range
        bucket_index = min(bucket_index, num_buckets - 1)
        # Assign the corresponding rating label to the score
        rating = rating_labels[bucket_index]
        rating_map[score] = rating
    
    return rating_map

In [20]:
create_rating_map(fico, 3)

{605: 'Very Good',
 572: 'Very Good',
 602: 'Very Good',
 612: 'Very Good',
 631: 'Very Good',
 697: 'Very Good',
 722: 'Good',
 545: 'Excellent',
 676: 'Very Good',
 447: 'Excellent',
 640: 'Very Good',
 636: 'Very Good',
 527: 'Excellent',
 638: 'Very Good',
 616: 'Very Good',
 644: 'Very Good',
 606: 'Very Good',
 727: 'Good',
 627: 'Very Good',
 581: 'Very Good',
 667: 'Very Good',
 480: 'Excellent',
 677: 'Very Good',
 726: 'Good',
 596: 'Very Good',
 642: 'Very Good',
 580: 'Very Good',
 542: 'Excellent',
 668: 'Very Good',
 787: 'Good',
 673: 'Very Good',
 609: 'Very Good',
 541: 'Excellent',
 560: 'Very Good',
 607: 'Very Good',
 746: 'Good',
 530: 'Excellent',
 615: 'Very Good',
 552: 'Excellent',
 599: 'Very Good',
 679: 'Very Good',
 603: 'Very Good',
 764: 'Good',
 736: 'Good',
 751: 'Good',
 610: 'Very Good',
 663: 'Very Good',
 737: 'Good',
 629: 'Very Good',
 689: 'Very Good',
 587: 'Very Good',
 692: 'Very Good',
 706: 'Good',
 515: 'Excellent',
 702: 'Very Good',
 583:

In [23]:

def quantize_fico_scores(fico_scores, num_buckets):
    min_score = min(fico_scores)
    max_score = max(fico_scores)
    bucket_width = (max_score - min_score) / num_buckets
    
    rating_map = {}
    for score in fico_scores:
        bucket_index = int((score - min_score) / bucket_width)
        rating_map[score] = bucket_index
    
    return rating_map

# Read loan data from CSV
df = pd.read_csv('Task 3 and 4_Loan_Data.csv')

x = df['default'].tolist()
y = df['fico_score'].tolist()

# Quantize FICO scores into rating buckets
rating_map = quantize_fico_scores(y, num_buckets=5)

# Calculate default and total counts for each rating bucket
default = [0 for _ in range(5)]
total = [0 for _ in range(5)]

for i in range(len(x)):
    fico_score = y[i]
    rating = rating_map[fico_score]
    default[rating] += x[i]
    total[rating] += 1

for i in range(1, 5):
    default[i] += default[i-1]
    total[i] += total[i-1]

# Log-likelihood calculation and dynamic programming
r = 10
dp = [[[-float('inf'), 0] for _ in range(5)] for _ in range(r+1)]

for i in range(r+1):
    for j in range(5):
        if i == 0:
            dp[i][j][0] = 0
        else:
            for k in range(j):
                if total[j] == total[k]:
                    continue
                if i == 1:
                    dp[i][j][0] = log_likelihood(total[j], default[j])
                else:
                    likelihood = log_likelihood(total[j]-total[k], default[j]-default[k])
                    if dp[i][j][0] < (dp[i-1][k][0] + likelihood):
                        dp[i][j][0] = likelihood + dp[i-1][k][0]
                        dp[i][j][1] = k

# Retrieve the FICO score buckets for the best rating map
k = 4
rating_buckets = []
while r >= 0:
    rating_buckets.append(k)
    k = dp[r][k][1]
    r -= 1

# Map the rating buckets to their respective FICO score ranges
min_score = min(y)
max_score = max(y)
bucket_width = (max_score - min_score) / 5

rating_ranges = []
for bucket in rating_buckets[::-1]:
    start_range = round(min_score + (bucket_width * bucket))
    end_range = round(min_score + (bucket_width * (bucket + 1)))
    rating_ranges.append((start_range, end_range))

print(rating_ranges)


IndexError: list index out of range